## Libraries

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

## Load features
To load the features you first have to create them, run the notebook feature_engineering. Beware it takes about 2-3 hours to run so save your features!

In [ ]:
train = pd.read_pickle('../features/train_new.pkl')
print(train.shape)

In [ ]:
test = pd.read_pickle('../features/test_new.pkl')
print(test.shape)

## Consts
Always use constant SEED otherwise the experiment is not reproducable, in that case why are we doing it? 

In [ ]:
SEED = 42
NUM_WORDS = 10000
SEQ_MAX_LEN = 40
EMBEDDING_DIM = 300 # 50, 100, 200 or 300

np.random.seed(SEED)
tf.set_random_seed(SEED)

## Feature selection

In [ ]:
from sklearn.feature_selection import VarianceThreshold
from keras.preprocessing import sequence, text

try:
    from sklearn.model_selection import train_test_split
except ImportError:
    from sklearn.cross_validation import train_test_split

In [ ]:
features = ['word_share', 'start_with_same_world', 'q1_char_num', 'q2_char_num',
       'q1_word_num', 'q2_word_num', 'rfidf_share', 'char_difference',
       'word_difference', 'seq_simhash_distance', 'shingle_simhash_distance',
       'avg_word_len_q1', 'avg_word_len_q2', 'avg_word_difference',
       'unigrams_common_count', 'bigrams_common_count',
       'unigrams_common_ratio', 'bigrams_common_ratio', 'cosin_sim',
       'word2vec_q1_mean', 'word2vec_q2_mean', 'q1_NN_count', 'q2_NN_count',
       'NN_diff', 'q1_RB_count', 'q2_RB_count', 'RB_diff', 'q1_VB_count',
       'q2_VB_count', 'VB_diff', 'q1_DT_count', 'q2_DT_count', 'DT_diff',
       'q1_JJ_count', 'q2_JJ_count', 'JJ_diff', 'q1_FW_count', 'q2_FW_count',
       'FW_diff', 'q1_RP_count', 'q2_RP_count', 'RP_diff', 'q1_SYM_count',
       'q2_SYM_count', 'SYM_diff']

target = 'is_duplicate'

In [ ]:
X = train[features]
y = train[target]

In [ ]:
tokenizer = text.Tokenizer(num_words=NUM_WORDS)
tokenizer.fit_on_texts(list(train.question1.values.astype(str)) + list(train.question2.values.astype(str)))

word_index = tokenizer.word_index
print('Found %d unique words in training set' % len(word_index))

In [ ]:
x1 = tokenizer.texts_to_sequences(train.question1.values.astype(str))
x1 = sequence.pad_sequences(x1, maxlen=SEQ_MAX_LEN)

x2 = tokenizer.texts_to_sequences(train.question2.values.astype(str))
x2 = sequence.pad_sequences(x2, maxlen=SEQ_MAX_LEN)

In [ ]:
X = np.array(X)
x1 = np.array(x1)
x2 = np.array(x2)
y = np.array(y)

In [ ]:
print(x1.shape, X.shape)

## Cross validation

In [ ]:
X_train, X_val, x1_train, x1_val, x2_train, x2_val, y_train, y_val = train_test_split(
    X, x1, x2, y, test_size=0.2, random_state=SEED)

print(X_train.shape, x1_train.shape, y_train.shape)

## Pretrained embeddings
Glove pretrained word2vec, source: https://nlp.stanford.edu/projects/glove/

Download: http://nlp.stanford.edu/data/glove.6B.zip

Use 100 dimensional vectors.

In [ ]:
import os

In [ ]:
embeddings_index = {}
with open('../pretrained/glove.6B/glove.6B.' + str(EMBEDDING_DIM) + 'd.txt', encoding='utf-8') as embedding_file:
    for line in embedding_file:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
        
print('Found %s pretrained word vectors.' % len(embeddings_index))

Create a embedding matrix, each row coresponds to a token (id for a word) and contains a word2vec for that word.

In [ ]:
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        
print(embedding_matrix.shape)

## Oversampling
Oversampling leads to local validation score not matching the score from public LB on kaggle. Models with oversampling usually perform a bit better, but due to scores not maching if possible better not use it.

The idea for oversampling came from Kaggle (https://www.kaggle.com/davidthaler/quora-question-pairs/how-many-1-s-are-in-the-public-lb) because the training and test set do not have the same distribution of dublicate questions. The train set has around 37% of duplicates while the private test set has 16.5% but the problem is that we only see the 35% of the prive test set. Final results are calculate on the remaining 65%, what if the distribution of the 35% set doe not match the other 65%, in that case oversampling while increasing the public LB score currently would yield in overfitting the score and poor results in the end.

In [ ]:
def oversample(X, y, rate=0.165):
    pos_train = X[y == 1]
    neg_train = X[y == 0]

    # Now we oversample the negative class
    # There is likely a much more elegant way to do this...
    p = 0.165
    scale = ((len(pos_train) / (len(pos_train) + len(neg_train))) / p) - 1
    while scale > 1:
        neg_train = pd.concat([neg_train, neg_train])
        scale -=1
    neg_train = pd.concat([neg_train, neg_train[:int(scale * len(neg_train))]])
    print(len(pos_train) / (len(pos_train) + len(neg_train)))

    X = pd.concat([pos_train, neg_train])
    y = (np.zeros(len(pos_train)) + 1).tolist() + np.zeros(len(neg_train)).tolist()

    return X, y

y_untouched = y_train

x1_train = pd.DataFrame(x1_train) 
x2_train = pd.DataFrame(x2_train) 
X_train = pd.DataFrame(X_train)

X_train, y_train = oversample(X_train, y_untouched)
x1_train, y_train = oversample(x1_train, y_untouched)
x2_train, y_train = oversample(x2_train, y_untouched)

X_train = np.array(X_train)
x1_train = np.array(x1_train)
x2_train = np.array(x2_train)

print(len(X_train), len(x1_train), len(x2_train), len(y_train))

## Normalization
Normalization helps but only if X is normalized, normalizing x1 and x2 does not allow the model to converge and pass the val_logloss of 0.42 -> bad. So far it seems that StandardScaler applied only on X does the trick.

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
scaler_X = StandardScaler()

X = scaler_X.fit_transform(X)

In [ ]:
# scaler_x1 = MinMaxScaler()
# x1 = scaler_x1.fit_transform(x1)

# scaler_x2 = MinMaxScaler()
# x2 = scaler_x2.fit_transform(x2)

## Model

In [ ]:
from sklearn.metrics import log_loss, auc, f1_score

from keras.layers.advanced_activations import PReLU
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU

from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping
from keras.layers import Merge

from keras.models import Sequential
from keras.layers import Activation, Dropout, Dense, BatchNormalization, TimeDistributed, Input
from keras.layers import MaxPooling1D, Lambda, Convolution1D, Flatten
from keras_tqdm import TQDMNotebookCallback

from keras.optimizers import Adam, RMSprop, Adamax, Adagrad, Nadam
from keras.activations import elu, relu

from keras.models import load_model

import keras
from keras import backend as K
K.set_image_dim_ordering('tf')

### Create base models

In [ ]:
model_q1 = Sequential()
model_q1.add(Embedding(len(word_index) + 1,
                       EMBEDDING_DIM,
                       weights=[embedding_matrix],
                       input_length=SEQ_MAX_LEN,
                       trainable=False))

model_q1.add(LSTM(128, recurrent_dropout=0., dropout=0., return_sequences=False))

model_q2 = Sequential()
model_q2.add(Embedding(len(word_index) + 1,
                       EMBEDDING_DIM,
                       weights=[embedding_matrix],
                       input_length=SEQ_MAX_LEN,
                       trainable=False))

model_q2.add(LSTM(128, recurrent_dropout=0., dropout=0., return_sequences=False))

model_GRU = Sequential()
model_GRU.add(Merge([model_q1, model_q2], mode = 'concat'))

model_GRU.add(Dense(256))
model_GRU.add(BatchNormalization())
model_GRU.add(Activation(relu))

model_GRU.add(Dense(256))
model_GRU.add(BatchNormalization())
model_GRU.add(Activation(relu))

model_GRU.add(Dense(1, activation='sigmoid'))

In [ ]:
model_glove1 = Sequential()
model_glove1.add(Embedding(len(word_index) + 1,
                     EMBEDDING_DIM,
                     weights=[embedding_matrix],
                     input_length=SEQ_MAX_LEN,
                     trainable=False))

model_glove1.add(TimeDistributed(Dense(100)))
model_glove1.add(BatchNormalization())
model_glove1.add(Activation(relu))
                 
model_glove1.add(Lambda(lambda x: K.sum(x, axis=1), output_shape=(100,)))

model_glove2 = Sequential()
model_glove2.add(Embedding(len(word_index) + 1,
                     EMBEDDING_DIM,
                     weights=[embedding_matrix],
                     input_length=SEQ_MAX_LEN,
                     trainable=False))

model_glove2.add(TimeDistributed(Dense(100)))
model_glove2.add(BatchNormalization())
model_glove2.add(Activation(relu))

model_glove2.add(Lambda(lambda x: K.sum(x, axis=1), output_shape=(100,)))

model_glove_embedding = Sequential()
model_glove_embedding.add(Merge([model_glove1, model_glove2], mode = 'concat'))

model_glove_embedding.add(Dense(32))
model_glove_embedding.add(BatchNormalization())
model_glove_embedding.add(Activation(relu))

model_glove_embedding.add(Dense(64))
model_glove_embedding.add(BatchNormalization())
model_glove_embedding.add(Activation(relu))

In [ ]:
model_glove_conv1 = Sequential()
model_glove_conv1.add(Embedding(len(word_index) + 1,
                     EMBEDDING_DIM,
                     weights=[embedding_matrix],
                     input_length=SEQ_MAX_LEN,
                     trainable=False))

model_glove_conv1.add(Convolution1D(filters = 64, kernel_size = 5, padding = 'same'))
model_glove_conv1.add(BatchNormalization())
model_glove_conv1.add(Activation(elu))
model_glove_conv1.add(MaxPooling1D(5))

model_glove_conv1.add(Convolution1D(filters = 128, kernel_size = 5, padding = 'same'))
model_glove_conv1.add(BatchNormalization())
model_glove_conv1.add(Activation(elu))
model_glove_conv1.add(MaxPooling1D(5))
model_glove_conv1.add(Dropout(0.2))

model_glove_conv1.add(Flatten())
model_glove_conv1.add(Dense(128))
model_glove_conv1.add(BatchNormalization())
model_glove_conv1.add(Activation(relu))

model_glove_conv2 = Sequential()
model_glove_conv2.add(Embedding(len(word_index) + 1,
                     EMBEDDING_DIM,
                     weights=[embedding_matrix],
                     input_length=SEQ_MAX_LEN,
                     trainable=False))

model_glove_conv2.add(Convolution1D(filters = 64, kernel_size = 5, padding = 'same'))
model_glove_conv2.add(BatchNormalization())
model_glove_conv2.add(Activation(relu))
model_glove_conv2.add(MaxPooling1D(5))

model_glove_conv2.add(Convolution1D(filters = 128, kernel_size = 5, padding = 'same'))
model_glove_conv2.add(BatchNormalization())
model_glove_conv2.add(Activation(relu))
model_glove_conv2.add(MaxPooling1D(5))
model_glove_conv2.add(Dropout(0.2))

model_glove_conv2.add(Flatten())
model_glove_conv2.add(Dense(128))
model_glove_conv2.add(BatchNormalization())
model_glove_conv2.add(Activation(relu))

model_glove_conv = Sequential()
model_glove_conv.add(Merge([model_glove_conv1, model_glove_conv2], mode = 'concat'))

model_glove_conv.add(Dense(32))
model_glove_conv.add(BatchNormalization())
model_glove_conv.add(Activation(relu))

model_glove_conv.add(Dense(64))
model_glove_conv.add(BatchNormalization())
model_glove_conv.add(Activation(relu))

In [ ]:
model_features = Sequential()
model_features.add(Dense(64, input_shape=(X.shape[1],)))
model_features.add(BatchNormalization())
model_features.add(Activation(relu))

model_features.add(Dense(128))
model_features.add(BatchNormalization())
model_features.add(Activation(relu))
model_features.add(Dropout(0.3))

model_features.add(Dense(512))
model_features.add(BatchNormalization())
model_features.add(Activation(relu))

In [ ]:
merged_model = Sequential()
merged_model.add(Merge([model_GRU, model_glove_embedding, model_glove_conv, model_features], mode = 'concat'))
merged_model.add(BatchNormalization())

merged_model.add(Dense(64))
merged_model.add(BatchNormalization())
merged_model.add(Activation(relu))
merged_model.add(Dropout(0.3))

merged_model.add(Dense(128))
merged_model.add(BatchNormalization())
merged_model.add(Activation(relu))

merged_model.add(Dense(1, activation='sigmoid'))

In [ ]:
model_GRU.compile(loss='binary_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [ ]:
#merged_model = load_model('TreeMergeNetOS.h5')

In [ ]:
# keras_logger = keras.callbacks.TensorBoard(log_dir='../notebooks/tensor_logs/mergnet5',
#                                            histogram_freq=1, write_graph=True, write_images=True)

# keras_logger.set_model(merged_model)

In [ ]:
callbacks = [ModelCheckpoint('GRU.h5', monitor='val_loss', save_best_only=True,
                             mode='auto', save_weights_only=True),
             EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto'),
             TQDMNotebookCallback()]

In [ ]:
train_data = [x1_train, x2_train, x1_train, x2_train, x1_train, x2_train, X_train]
val_data = ([x1_val, x2_val, x1_val, x2_val, x1_val, x2_val, X_val], y_val)

In [ ]:
model_GRU.fit([x1_train, x2_train],
          y_train,
          batch_size=256 * 2,
          epochs=50,
          verbose=0,
          validation_data=([x1_val, x2_val], y_val),
          callbacks=callbacks)

In [ ]:
model_GRU.save('GRU.h5')

Epoch 0:
[loss: 0.567, acc: 0.703, val_loss: 0.510, val_acc: 0.745] 100% 323432/323432 [04:37<00:00, 82.31it/s]
Epoch 1:
[loss: 0.495, acc: 0.754, val_loss: 0.470, val_acc: 0.767] 100% 323432/323432 [04:44<00:00, 83.35it/s]
Epoch 2:
[loss: 0.464, acc: 0.772, val_loss: 0.449, val_acc: 0.779] 100% 323432/323432 [04:44<00:00, 84.52it/s]
Epoch 3:
[loss: 0.443, acc: 0.785, val_loss: 0.438, val_acc: 0.788] 100% 323432/323432 [04:45<00:00, 84.70it/s]
Epoch 4:
[loss: 0.425, acc: 0.795, val_loss: 0.427, val_acc: 0.794] 100% 323432/323432 [04:37<00:00, 92.55it/s]
Epoch 5:
[loss: 0.412, acc: 0.804, val_loss: 0.425, val_acc: 0.797] 100% 323432/323432 [04:37<00:00, 90.10it/s]
Epoch 6:
[loss: 0.399, acc: 0.811, val_loss: 0.419, val_acc: 0.800] 100% 323432/323432 [04:38<00:00, 92.76it/s]
Epoch 7:
[loss: 0.388, acc: 0.817, val_loss: 0.414, val_acc: 0.804] 100% 323432/323432 [04:37<00:00, 92.67it/s]
Epoch 8:
[loss: 0.379, acc: 0.822, val_loss: 0.413, val_acc: 0.805] 100% 323432/323432 [04:43<00:00, 81.17it/s]
Epoch 9:
[loss: 0.370, acc: 0.828, val_loss: 0.410, val_acc: 0.809] 100% 323432/323432 [04:43<00:00, 83.42it/s]
Epoch 10:
[loss: 0.362, acc: 0.831, val_loss: 0.412, val_acc: 0.808] 100% 323432/323432 [04:39<00:00, 93.99it/s]
Epoch 11:
[loss: 0.353, acc: 0.836, val_loss: 0.419, val_acc: 0.801] 100% 323432/323432 [04:38<00:00, 95.13it/s]

In [ ]:
#merged_model.save('TreeMergeNetOS.h5')

In [ ]:
# scores = model.evaluate(X, y, verbose=0, batch_size=4096 * 8)

# print("Model validation accuracy: %.2f" % (scores[1]*100))
# print("Model validation loss: %.4f" % (scores[0]))

## Generate submission

Chunker is used to lower RAM requirements, without chunking requirement goes up to about 24GB of RAM.

In [ ]:
import math

def chunker(collection, chunk_size=130000):
    chunk_num = math.ceil(collection.shape[0] / float(chunk_size))
    for i in range(chunk_num):
        yield collection[chunk_size*i : chunk_size*(i+1)]

In [ ]:
#X_test = test[features]

In [ ]:
preds = []
for q1, q2, test_row in zip(
    chunker(test.question1), chunker(test.question2), chunker(test)
):
    print('%d / %d' % (len(preds), len(test)))
    x1_test_row = tokenizer.texts_to_sequences(q1.values.astype(str))
    x1_test_row = sequence.pad_sequences(x1_test_row, maxlen=SEQ_MAX_LEN)

    x2_test_row = tokenizer.texts_to_sequences(q2.values.astype(str))
    x2_test_row = sequence.pad_sequences(x2_test_row, maxlen=SEQ_MAX_LEN)
    
    #x1_test_row = scaler_x1.transform(x1_test_row)
    #x2_test_row = scaler_x2.transform(x2_test_row)
    X_test_row = test_row[features]
    X_test_row = scaler_X.transform(X_test_row)

    batch_preds = merged_model.predict([x1_test_row, x2_test_row, x1_test_row,
                                        x2_test_row, x1_test_row, x2_test_row, X_test_row],
                                       batch_size=128 * 8)
    preds.extend(batch_preds)

In [ ]:
print(len(preds))

In [ ]:
np.savetxt(
    '../submissions/submission.csv', np.c_[range(len(preds)), preds],
    delimiter=',', header='test_id,is_duplicate', comments='', fmt='%d,%f'
)

## Model summary

In [ ]:
merged_model.summary()